# Ejercicios Numpy y DataFrame Parte 3

Importamos las librerías

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Ejercicio 1.

Simular operaciones sobre un inventario con pandas y NumPy.

1. Calcular el valor total por producto (stock * precio).
2. Usar NumPy para subir todos los precios un 10%.
3. Filtrar productos con stock menor a 20.
4. Crear una función de Python que clasifique los productos como:
   * "bajo" si el stock < 20
   * "medio" si está entre 20 y 50
   * "alto" si es > 50

In [11]:
productos = pd.DataFrame({
    "producto": ["Monitor", "Teclado", "Mouse", "Parlantes", "Webcam"],
    "stock": [25, 50, 80, 15, 30],
    "precio": [150.0, 20.0, 15.0, 45.0, 60.0]
})

def clasificar_stock(cantidad):
    if cantidad < 20:
        return "bajo"
    elif 20 <= cantidad <= 50:
        return "medio"
    else:
        return "alto"

# Calculamos la inversión
productos["gasto"] = productos["stock"] * productos["stock"]
print("\nGastos:\n",productos)

# Calculamos el aumento del 10%
# productos["precio"] = productos["precio"].apply(lambda x: x * 1.1)
productos["precio"] = productos["precio"] * 1.1
print("\nIncremento:\n",productos)

# Productos con menor stock a 20.
filtro = productos["stock"] <20
print("\nStock mayor a 20:\n",productos[filtro])

#
productos["nivel_stock"] = productos["stock"].apply(clasificar_stock)
print(productos)


Gastos:
     producto  stock  precio  gasto
0    Monitor     25   150.0    625
1    Teclado     50    20.0   2500
2      Mouse     80    15.0   6400
3  Parlantes     15    45.0    225
4     Webcam     30    60.0    900

Incremento:
     producto  stock  precio  gasto
0    Monitor     25   165.0    625
1    Teclado     50    22.0   2500
2      Mouse     80    16.5   6400
3  Parlantes     15    49.5    225
4     Webcam     30    66.0    900

Stock mayor a 20:
     producto  stock  precio  gasto
3  Parlantes     15    49.5    225
    producto  stock  precio  gasto nivel_stock
0    Monitor     25   165.0    625       medio
1    Teclado     50    22.0   2500       medio
2      Mouse     80    16.5   6400        alto
3  Parlantes     15    49.5    225        bajo
4     Webcam     30    66.0    900       medio


### Ejericio 2

Evaluar medidas científicas usando lógica Python + análisis con pandas + transformaciones NumPy.

* Calcular el promedio de cada experimento.
* Usar NumPy para detectar valores fuera del rango 40–60.
* Reemplazar valores < 45 por la media del experimento.
* Crear una columna de “evaluación” por día: "estable" si std < 5, "inestable" si no.

In [10]:
# Simulamos resultados de 5 experimentos en 4 días
np.random.seed(1)

# Generamos números aleatorios, con media 50 y desviación de 5 en una matriz de 4x5
resultados = np.random.normal(loc=50, scale=5, size=(4, 5))

# Creamos nuestro data frame
df = pd.DataFrame(resultados, columns=[f"Exp_{i}" for i in range(1, 6)],
                  index=["Día 1", "Día 2", "Día 3", "Día 4"])

# Agregar una fila al dataframe para su promedio
promedios = df.mean()
print("Promedio por experimento:\n", promedios)

# Valores que esten fuera del rango deseado.
fuera_de_rango = (df.values < 40) | (df.values > 60)
print("Valores fuera de rango (True/False):\n", fuera_de_rango)

# Corregimos los valores que esten abajo de 45.
df_corregido = df.copy()

# Iteramos sobre las columnas.
for col in df.columns:
    media_exp = df[col].mean()
    df_corregido[col] = df[col].apply(lambda x: media_exp if x < 45 else x)

print("\nValores corregidos (reemplazados si < 45):\n", df_corregido)

# Calculamos la desviación por día.
df_corregido["evaluación"] = df_corregido.std(axis=1).apply(
    lambda std: "estable" if std < 5 else "inestable"
)

print("\nEvaluación por día (basado en std):\n", df_corregido["evaluación"])

Promedio por experimento:
 Exp_1    49.606279
Exp_2    48.625608
Exp_3    46.887932
Exp_4    48.630287
Exp_5    52.915777
dtype: float64
Valores fuera de rango (True/False):
 [[False False False False False]
 [ True False False False False]
 [False  True False False False]
 [False False False False False]]

Valores corregidos (reemplazados si < 45):
            Exp_1      Exp_2      Exp_3      Exp_4      Exp_5
Día 1  58.121727  46.941218  47.359141  48.630287  54.327038
Día 2  49.606279  58.724059  46.193965  51.595195  48.753148
Día 3  57.310540  48.625608  48.387914  48.079728  55.668847
Día 4  49.606279  49.137859  45.610708  50.211069  52.914076

Evaluación por día (basado en std):
 Día 1    estable
Día 2    estable
Día 3    estable
Día 4    estable
Name: evaluación, dtype: object


### Ejercicio 3

Unir condiciones lógicas de Python, agrupamientos y cálculos en DataFrames.
1. Calcular el total de ventas por departamento.
2. Crear una nueva columna "bono":
    * 5% de ventas si hizo > 5 horas extra
    * 3% en caso contrario
3. Clasificar empleados como "destacado" si hizo > 10k en ventas y > 5 horas extra.
4. Calcular el promedio de ventas por grupo usando groupby.

In [26]:
empleados = pd.DataFrame({
    "nombre": ["Ana", "Luis", "María", "Carlos", "Elena", "Pablo", "Lucía", "Andrés"],
    "departamento": ["Ventas", "TI", "Ventas", "TI", "Recursos Humanos", "Ventas", "TI", "Recursos Humanos"],
    "ventas_mensuales": [12000, 15000, 9000, 13000, 5000, 11000, 14000, 4500],
    "horas_extra": [5, 10, 2, 8, 1, 6, 9, 0]
})

# Ventas totales por departamento.
ventasProm = empleados.groupby("departamento")["ventas_mensuales"].mean().reset_index()
print("Ventas Promedio por Departamento:\n", ventasProm)

# Columna Bono
empleados["bono"] = empleados.apply(
    lambda fila: fila["ventas_mensuales"] * 0.05 if fila["horas_extra"] > 5 else fila["ventas_mensuales"] * 0.03,
    axis=1
)

# Calcular a los empleados destacados
empleados["status"] = empleados.apply(
    lambda fila: "destacado" if fila["ventas_mensuales"] >10000 and fila["horas_extra"] > 5 else "normal",
    axis=1
)
empleados

Ventas Promedio por Departamento:
        departamento  ventas_mensuales
0  Recursos Humanos       4750.000000
1                TI      14000.000000
2            Ventas      10666.666667


,nombre,departamento,ventas_mensuales,horas_extra,bono,status
0,Ana,Ventas,12000,5,360.0,normal
1,Luis,TI,15000,10,750.0,destacado
2,María,Ventas,9000,2,270.0,normal
3,Carlos,TI,13000,8,650.0,destacado
4,Elena,Recursos Humanos,5000,1,150.0,normal
5,Pablo,Ventas,11000,6,550.0,destacado
6,Lucía,TI,14000,9,700.0,destacado
7,Andrés,Recursos Humanos,4500,0,135.0,normal


In [21]:
empleados

,nombre,departamento,ventas_mensuales,horas_extra
0,Ana,Ventas,12000,5
1,Luis,TI,15000,10
2,María,Ventas,9000,2
3,Carlos,TI,13000,8
4,Elena,Recursos Humanos,5000,1
5,Pablo,Ventas,11000,6
6,Lucía,TI,14000,9
7,Andrés,Recursos Humanos,4500,0
